In [1]:
import requests
import itertools
import copy
import os
import yaml
import xarray as xr
import numpy as np
from variable_mapping import variable_mapping
import datetime
import csv
import sys
import time
import pandas as pd
import re
from dotenv import dotenv_values
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

from providentia.auxiliar import CURRENT_PATH, join

PROVIDENTIA_ROOT = '/home/avilanov/software/Providentia/'
CURRENT_PATH = os.getcwd()
sys.path = [path for path in sys.path if '../dependencies/GHOST_standards/' not in path]            
sys.path.insert(1, os.path.join(CURRENT_PATH, '../dependencies/GHOST_standards/1.5'))
from GHOST_standards import standard_parameters, get_standard_metadata

In [2]:
start_time = time.time()

In [3]:
variables = ['sconcno2']
resolution = 'daily'

#variables = ['sconco3']
#resolution = 'hourly'
target_start_date = datetime.datetime(2018, 1, 1, 0)
target_end_date = datetime.datetime(2018, 12, 31, 23)

In [4]:
coverages_dict = {'P0000-00-00T01:00:00': 'hourly', 
                  'P0000-00-01T00:00:00': 'daily', 
                  'P0000-01-00T00:00:00': 'monthly'}

units_dict = {'nmol/mol': 'nmol_per_mol',
              'ug/m3': 'ug_per_m3',
              'ug N/m3': 'ug_N_per_m3',
              'ug S/m3': 'ug_S_per_m3',
              'pmol/mol': 'pmol_per_mol',
              '1/Mm': '1_per_Mm'
              } 

# Get ACTRIS variable mapping

In [5]:
def create_variable_mapping_file():
    result = {
        value['preferred_term'].replace('"', ''): {'var': key[2], 'units': key[0]}
        for key, value in variable_mapping.items()
    }
    
    with open('variable_mapping.yaml', 'w') as file:
        yaml.dump(result, file, default_flow_style=False)

In [6]:
#create_variable_mapping_file()

In [7]:
variable_mapping = yaml.safe_load(open(os.path.join(CURRENT_PATH, 'variable_mapping.yaml')))
variable_mapping = {k: v for k, v in variable_mapping.items() if k.strip() and v}

# Get BSC-ACTRIS parameters dictionary

In [8]:
def create_actris_variables_file():
    with open('actris_variables.csv', mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        for key in variable_mapping.keys():
            writer.writerow([key, variable_mapping[key]['var']])

In [9]:
#create_actris_variables_file()

In [10]:
def create_ghost_variables_file():    
    with open('ghost_variables.csv', mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        for key in standard_parameters.keys():
            writer.writerow([standard_parameters[key]['long_parameter_name'], standard_parameters[key]['bsc_parameter_name'], ', '.join( standard_parameters[key]['ebas_parameter_name'])])

In [11]:
#create_ghost_variables_file()

In [12]:
parameters_dict = {}
with open('ghost_actris_variables.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        # if ACTRIS was found (manual intervention)
        if len(row[6]) != 0 and row[6] != 'Preferred':
            # 'Preferred' = 'BSC'
            parameters_dict[row[1]] = row[6]
parameters_dict

{'sconco3': 'ozone mass concentration',
 'sconcno2': 'nitrogen dioxide mass concentration',
 'sconcso2': 'sulfur dioxide mass concentration',
 'sconcco': 'carbon monoxide amount fraction',
 'sconcch4': 'methane amount fraction',
 'sconcglyox': 'glyoxal mass concentration',
 'sconcc2h4': 'ethene amount fraction',
 'sconcald2': 'acetaldehyde mass concentration',
 'sconcc2h6': 'ethane amount fraction',
 'sconcetoh': 'ethanol mass concentration',
 'sconcc3h6': 'propene amount fraction',
 'sconcc3h8': 'propane amount fraction',
 'sconcc4h6': '1,3-butadiene amount fraction',
 'sconcc4h8': '1-butene amount fraction',
 'sconcisop': 'isoprene amount fraction',
 'sconcc5h12': 'n-pentane amount fraction',
 'sconcc6h6': 'benzene amount fraction',
 'sconcc6h14': 'n-hexane amount fraction',
 'sconcc7h8': 'toluene',
 'sconcmpxyl': 'm/p-xylenes amount fraction',
 'sconcmxyl': 'm-xylene amount fraction',
 'sconcoxyl': 'o-xylene amount fraction',
 'sconcc9h12': '1,2,4-trimethylbenzene amount fraction',


# Get BSC-ACTRIS metadata dictionary

In [13]:
standard_metadata = get_standard_metadata({'standard_units': ''})
ghost_metadata = list(standard_metadata.keys())

In [14]:
actris_metadata = ['Conventions', 'featureType', 'title', 'keywords', 'id', 'naming_authority',
                   'project', 'acknowledgement', 'doi', 'license', 'citation', 'summary', 'source', 
                   'institution', 'processing_level', 'date_created', 'date_metadata_modified', 
                   'creator_name', 'creator_type', 'creator_email', 'creator_institution',
                   'contributor_name', 'contributor_role', 'publisher_type', 'publisher_name', 
                   'publisher_institution', 'publisher_email', 'publisher_url', 'geospatial_bounds', 
                   'geospatial_bounds_crs', 'geospatial_lat_min', 'geospatial_lat_max',
                   'geospatial_lon_min', 'geospatial_lon_max', 'geospatial_vertical_min', 
                   'geospatial_vertical_max', 'geospatial_vertical_positive', 'time_coverage_start',
                   'time_coverage_end', 'time_coverage_duration', 'time_coverage_resolution',
                   'timezone', 'ebas_data_definition', 'ebas_data_license', 'ebas_citation', 
                   'ebas_set_type_code', 'ebas_timezone', 'ebas_file_name', 'ebas_represents_doi',
                   'ebas_contains_doi', 'ebas_file_creation', 'ebas_export_state', 'ebas_export_filter',
                   'ebas_startdate', 'ebas_revision_date', 'ebas_data_level', 'ebas_period_code',
                   'ebas_resolution_code', 'ebas_sample_duration', 'ebas_orig_time_res',
                   'ebas_station_code', 'ebas_platform_code', 'ebas_station_name', 
                   'ebas_station_wdca_id', 'ebas_station_gaw_id', 'ebas_station_gaw_name',
                   'ebas_station_land_use', 'ebas_station_setting', 'ebas_station_gaw_type', 
                   'ebas_station_wmo_region', 'ebas_station_latitude', 'ebas_station_longitude', 
                   'ebas_station_altitude', 'ebas_measurement_height', 'ebas_regime', 'ebas_component',
                   'ebas_matrix', 'ebas_laboratory_code', 'ebas_instrument_type', 'ebas_instrument_name',
                   'ebas_instrument_manufacturer', 'ebas_instrument_model', 
                   'ebas_instrument_serial_number', 'ebas_method_ref', 'ebas_standard_method',
                   'ebas_inlet_type', 'ebas_inlet_description', 'ebas_humidity_temperaure_control',
                   'ebas_absorption_cross_section', 'ebas_organization', 'ebas_framework_acronym',
                   'ebas_framework_name', 'ebas_framework_description', 'ebas_framework_contact_name',
                   'ebas_framework_contact_email', 'ebas_originator', 'ebas_submitter', 
                   'ebas_acknowledgement', 'Metadata_Conventions', 'geospatial_lat_units', 
                   'geospatial_lon_units', 'comment', 'standard_name_vocabulary', 'history', 
                   'creator_url']

In [15]:
all_metadata_dict = {'station_reference': 'ebas_station_code',
                     'WIGOS_station_identifier': '',
                     'station_timezone': 'timezone',
                     'latitude': 'ebas_station_latitude',
                     'longitude': 'ebas_station_longitude',
                     'altitude': 'ebas_station_altitude',
                     'sampling_height': 'ebas_measurement_height',
                     'measurement_altitude': 'ebas_station_altitude', # no diff with altitude?
                     'ellipsoid': '',
                     'horizontal_datum': '',
                     'vertical_datum': '',
                     'projection': '',
                     'distance_to_building': '',
                     'distance_to_kerb': '',
                     'distance_to_junction': '',
                     'distance_to_source': '',
                     'street_width': '',
                     'street_type': '',
                     'daytime_traffic_speed': '',
                     'daily_passing_vehicles': '',
                     'data_level': 'ebas_data_level',
                     'climatology': '',
                     'station_name': 'ebas_station_name',
                     'city': '', # get from title
                     'country': '',
                     'administrative_country_division_1': '',
                     'administrative_country_division_2': '',
                     'population': '',
                     'representative_radius': '',
                     'network': 'naming_authority',
                     'associated_networks': 'naming_authority',
                     'area_classification': '',
                     'station_classification': '',
                     'main_emission_source': '',
                     'land_use': 'ebas_station_land_use',
                     'terrain': '',
                     'measurement_scale': '',
                     'ESDAC_Iwahashi_landform_classification': '',
                     'ESDAC_modal_Iwahashi_landform_classification_5km': '',
                     'ESDAC_modal_Iwahashi_landform_classification_25km': '',
                     'ESDAC_Meybeck_landform_classification': '',
                     'ESDAC_modal_Meybeck_landform_classification_5km': '',
                     'ESDAC_modal_Meybeck_landform_classification_25km': '',
                     'GHSL_settlement_model_classification': '',
                     'GHSL_modal_settlement_model_classification_5km': '',
                     'GHSL_modal_settlement_model_classification_25km': '',
                     'Joly-Peuch_classification_code': '',
                     'Koppen-Geiger_classification': '',
                     'Koppen-Geiger_modal_classification_5km': '',
                     'Koppen-Geiger_modal_classification_25km': '',
                     'MODIS_MCD12C1_v6_IGBP_land_use': '',
                     'MODIS_MCD12C1_v6_modal_IGBP_land_use_5km': '',
                     'MODIS_MCD12C1_v6_modal_IGBP_land_use_25km': '',
                     'MODIS_MCD12C1_v6_UMD_land_use': '',
                     'MODIS_MCD12C1_v6_modal_UMD_land_use_5km': '',
                     'MODIS_MCD12C1_v6_modal_UMD_land_use_25km': '',
                     'MODIS_MCD12C1_v6_LAI': '',
                     'MODIS_MCD12C1_v6_modal_LAI_5km': '',
                     'MODIS_MCD12C1_v6_modal_LAI_25km': '',
                     'WMO_region': 'ebas_station_wmo_region',
                     'WWF_TEOW_terrestrial_ecoregion': '',
                     'WWF_TEOW_biogeographical_realm': '',
                     'WWF_TEOW_biome': '',
                     'UMBC_anthrome_classification': '',
                     'UMBC_modal_anthrome_classification_5km': '',
                     'UMBC_modal_anthrome_classification_25km': '',
                     'EDGAR_v4.3.2_annual_average_BC_emissions': '',
                     'EDGAR_v4.3.2_annual_average_CO_emissions': '',
                     'EDGAR_v4.3.2_annual_average_NH3_emissions': '',
                     'EDGAR_v4.3.2_annual_average_NMVOC_emissions': '',
                     'EDGAR_v4.3.2_annual_average_NOx_emissions': '',
                     'EDGAR_v4.3.2_annual_average_OC_emissions': '',
                     'EDGAR_v4.3.2_annual_average_PM10_emissions': '',
                     'EDGAR_v4.3.2_annual_average_biogenic_PM2.5_emissions': '',
                     'EDGAR_v4.3.2_annual_average_fossilfuel_PM2.5_emissions': '',
                     'EDGAR_v4.3.2_annual_average_SO2_emissions': '',
                     'ASTER_v3_altitude': '',
                     'ETOPO1_altitude': '',
                     'ETOPO1_max_altitude_difference_5km': '',
                     'GHSL_built_up_area_density': '',
                     'GHSL_average_built_up_area_density_5km': '',
                     'GHSL_average_built_up_area_density_25km': '',
                     'GHSL_max_built_up_area_density_5km': '',
                     'GHSL_max_built_up_area_density_25km': '',
                     'GHSL_population_density': '',
                     'GHSL_average_population_density_5km': '',
                     'GHSL_average_population_density_25km': '',
                     'GHSL_max_population_density_5km': '',
                     'GHSL_max_population_density_25km': '',
                     'GPW_population_density': '',
                     'GPW_average_population_density_5km': '',
                     'GPW_average_population_density_25km': '',
                     'GPW_max_population_density_5km': '',
                     'GPW_max_population_density_25km': '',
                     'NOAA-DMSP-OLS_v4_nighttime_stable_lights': '',
                     'NOAA-DMSP-OLS_v4_average_nighttime_stable_lights_5km': '',
                     'NOAA-DMSP-OLS_v4_average_nighttime_stable_lights_25km': '',
                     'NOAA-DMSP-OLS_v4_max_nighttime_stable_lights_5km': '',
                     'NOAA-DMSP-OLS_v4_max_nighttime_stable_lights_25km': '',
                     'OMI_level3_column_annual_average_NO2': '',
                     'OMI_level3_column_cloud_screened_annual_average_NO2': '',
                     'OMI_level3_tropospheric_column_annual_average_NO2': '',
                     'OMI_level3_tropospheric_column_cloud_screened_annual_average_NO2': '',
                     'GSFC_coastline_proximity': '',
                     'primary_sampling_type': '',
                     'primary_sampling_instrument_name': '',
                     'primary_sampling_instrument_documented_flow_rate': '',
                     'primary_sampling_instrument_reported_flow_rate': '',
                     'primary_sampling_process_details': '',
                     'primary_sampling_instrument_manual_name': '',
                     'primary_sampling_further_details': '',
                     'sample_preparation_types': '',
                     'sample_preparation_techniques': '',
                     'sample_preparation_process_details': '',
                     'sample_preparation_further_details': '',
                     'measurement_methodology': 'ebas_method_ref',
                     'measuring_instrument_name': 'ebas_instrument_name',
                     'measuring_instrument_sampling_type': 'ebas_instrument_type',
                     'measuring_instrument_documented_flow_rate': '',
                     'measuring_instrument_reported_flow_rate': '',
                     'measuring_instrument_process_details': '',
                     'measuring_instrument_manual_name': '',
                     'measuring_instrument_further_details': '',
                     'measuring_instrument_reported_units': '',
                     'measuring_instrument_reported_lower_limit_of_detection': '',
                     'measuring_instrument_documented_lower_limit_of_detection': '',
                     'measuring_instrument_reported_upper_limit_of_detection': '',
                     'measuring_instrument_documented_upper_limit_of_detection': '',
                     'measuring_instrument_reported_uncertainty': '',
                     'measuring_instrument_documented_uncertainty': '',
                     'measuring_instrument_reported_accuracy': '',
                     'measuring_instrument_documented_accuracy': '',
                     'measuring_instrument_reported_precision': '',
                     'measuring_instrument_documented_precision': '',
                     'measuring_instrument_reported_zero_drift': '',
                     'measuring_instrument_documented_zero_drift': '',
                     'measuring_instrument_reported_span_drift': '',
                     'measuring_instrument_documented_span_drift': '',
                     'measuring_instrument_reported_zonal_drift': '',
                     'measuring_instrument_documented_zonal_drift': '',
                     'measuring_instrument_reported_measurement_resolution': '',
                     'measuring_instrument_documented_measurement_resolution': '',
                     'measuring_instrument_reported_absorption_cross_section': '',
                     'measuring_instrument_documented_absorption_cross_section': '',
                     'measuring_instrument_inlet_information': 'ebas_inlet_description',
                     'measuring_instrument_calibration_scale': '',
                     'network_provided_volume_standard_temperature': '',
                     'network_provided_volume_standard_pressure': '',
                     'retrieval_algorithm': '',
                     'principal_investigator_name': 'creator_name',
                     'principal_investigator_institution': 'creator_institution',
                     'principal_investigator_email_address': 'creator_email',
                     'contact_name': 'publisher_name', # TODO: in doubt
                     'contact_institution': 'publisher_institution', # TODO: in doubt
                     'contact_email_address': 'publisher_email', # TODO: in doubt
                     'meta_update_stamp': 'date_metadata_modified',
                     'data_download_stamp': '', # TODO: Do we put our download date here?
                     'data_revision_stamp': '',
                     'network_sampling_details': '',
                     'network_uncertainty_details': '',
                     'network_maintenance_details': '',
                     'network_qa_details': '',
                     'network_miscellaneous_details': '',
                     'data_licence': 'license',
                     'process_warnings': ''}

In [16]:
metadata_dict = {k: v for k, v in all_metadata_dict.items() if v != ''}
metadata_dict

{'station_reference': 'ebas_station_code',
 'station_timezone': 'timezone',
 'latitude': 'ebas_station_latitude',
 'longitude': 'ebas_station_longitude',
 'altitude': 'ebas_station_altitude',
 'sampling_height': 'ebas_measurement_height',
 'measurement_altitude': 'ebas_station_altitude',
 'data_level': 'ebas_data_level',
 'station_name': 'ebas_station_name',
 'network': 'naming_authority',
 'associated_networks': 'naming_authority',
 'land_use': 'ebas_station_land_use',
 'WMO_region': 'ebas_station_wmo_region',
 'measurement_methodology': 'ebas_method_ref',
 'measuring_instrument_name': 'ebas_instrument_name',
 'measuring_instrument_sampling_type': 'ebas_instrument_type',
 'measuring_instrument_inlet_information': 'ebas_inlet_description',
 'principal_investigator_name': 'creator_name',
 'principal_investigator_institution': 'creator_institution',
 'principal_investigator_email_address': 'creator_email',
 'contact_name': 'publisher_name',
 'contact_institution': 'publisher_institution'

# Get files per variable

In [17]:
def get_files_per_var(var):
    files_per_var = {}
    base_url = "https://prod-actris-md.nilu.no/metadata/content"

    if var not in files_per_var:
        files_per_var[var] = {}

    variable_files = []
    page = 0
    while True:
        # set up URL with pagination
        url = f"{base_url}/{parameters_dict[var]}/page/{page}"
        response = requests.get(url)

        # check if the response is valid and contains data
        if response.status_code != 200:
            print(
                f"Error fetching page {page}. Status code: {response.status_code}")
            break

        data = response.json()

        # check if there's content in the data
        if not data:
            break

        # loop through each entry in the data and get OPeNDAP URL
        for item in data:
            doi = item.get("md_identification", {}).get(
                "identifier", {}).get("pid")
            opendap_urls = [protocol_dict['dataset_url'] for protocol_dict in item.get(
                'md_distribution_information', []) if protocol_dict.get('protocol') == 'OPeNDAP']

            # print DOI and OPeNDAP URL if both are present
            if doi and opendap_urls:
                variable_files.append(opendap_urls)

        # go to the next page
        page += 1

    files_per_var[var]['files'] = list(
        itertools.chain.from_iterable(variable_files))

    return files_per_var

# Get information on files

In [18]:
def get_files_path(var):

    alpha_var = ''.join(x for x in var if x.isalpha())
    if alpha_var in ['lsco', 'absco', 'lbsco', 'odaero']:
        path = join(PROVIDENTIA_ROOT, 'settings', 'internal', 'actris', f'files/{alpha_var}/files.yaml')
    else:
        path = join(PROVIDENTIA_ROOT, 'settings', 'internal', 'actris', f'files/{var}/files.yaml')

    return path

# Format data

In [19]:
def temporally_average_data(combined_ds, resolution, year, month, var):

    # get valid dates frequency
    if resolution == 'hourly':
        frequency = 'h'
    elif resolution == 'daily':
        frequency = 'D'
    elif resolution == 'monthly':
        frequency = 'MS'

    # get start and end of period to construct valid dates
    time = combined_ds.time.values
    start_date = datetime.datetime(year, month, 1)
    first_day_next_month = datetime.datetime(year, month % 12 + 1, 1) if month != 12 else datetime.datetime(year + 1, 1, 1)
    end_date = first_day_next_month - datetime.timedelta(days=1)
    valid_dates = pd.date_range(start=start_date, end=end_date, freq=frequency).to_numpy(dtype='datetime64[ns]')
    
    # initialise averaged data
    averaged_data = np.empty((len(combined_ds.station.values), len(valid_dates)))
    
    for station_i, station in enumerate(combined_ds.station.values):
        # initialise averaged data
        station_averaged_data = []
    
        # read data per station
        data = combined_ds[var].isel(station=station_i).values

        # ignore data (times and values) if the values are nan
        valid_idxs = ~np.isnan(data)
        valid_time = time[valid_idxs]
        valid_data = data[valid_idxs]

        # calculate weighted averages
        if len(valid_data) != 0:
            for date in valid_dates:
            
                # get differences between valid time and actual times in minutes
                time_diffs = (valid_time - date).astype('timedelta64[ns]').astype(float)
            
                # get positive differences and negative differences to differentiate 
                # between the actual times that are earlier than the valid date (negative), and those that are later (positive)
                positive_diffs = time_diffs[time_diffs > 0]
                negative_diffs = time_diffs[time_diffs < 0]
                
                # find the closest actual time after the valid time
                closest_positive = None
                if len(positive_diffs) > 0:
                    closest_positive_idx = np.abs(positive_diffs).argmin()
                    closest_positive = positive_diffs[closest_positive_idx]
                    closest_positive_time = valid_time[time_diffs == positive_diffs[closest_positive_idx]][0]
                    closest_positive_value = valid_data[time_diffs == positive_diffs[closest_positive_idx]][0]
            
                # find the closest actual time before the valid time
                closest_negative = None
                if len(negative_diffs) > 0:
                    closest_negative_idx = np.abs(negative_diffs).argmin()
                    closest_negative = negative_diffs[closest_negative_idx]
                    closest_negative_time = valid_time[time_diffs == negative_diffs[closest_negative_idx]][-1]
                    closest_negative_value = valid_data[time_diffs == negative_diffs[closest_negative_idx]][-1]
            
                # when the valid time only has a value in one direction, get closest value without calculating weights
                if closest_positive is None:
                    value = closest_negative_value
                elif closest_negative is None:
                    value = closest_positive_value
                # in the rest of cases, calculate weights of 2 closest values and make average
                else:
                    # get 2 closest times and make positive to be able to compare differences
                    closest_diffs = np.abs([closest_negative, closest_positive])
            
                    # we do the reverse, since we want the differences in minutes to have a heavier weight if these are smaller (nearer the actual time)
                    weights = 1 / closest_diffs
                    
                    # finally we normalize them to have values between 0 and 1
                    weights_normalized = weights / np.sum(weights)
            
                    # get average
                    value = np.average([closest_negative_value, closest_positive_value], weights=weights_normalized)
        
                # save averaged data
                station_averaged_data.append(value)
        
            averaged_data[station_i, :] = station_averaged_data
        else:
            averaged_data[station_i, :] = [np.nan]*len(valid_dates)
    
    # create new variable with averaged data
    combined_averaged_ds = xr.DataArray(
        data=averaged_data,
        coords={'station': combined_ds.station.values, 'time': valid_dates}, 
        dims=['station', 'time'],
        attrs={'units': combined_ds[var].units})
    
    # drop old variable and associated time
    combined_ds = combined_ds.drop_vars(var)
    combined_ds = combined_ds.drop_dims('time')
    
    # add new variable
    combined_ds[var] = combined_averaged_ds
    
    return combined_ds

In [20]:
def is_wavelength_var(actris_parameter):
    wavelength_var = False
    if actris_parameter in ['aerosol particle light absorption coefficient',
                            'aerosol particle light hemispheric backscatter coefficient',
                            'aerosol particle light scattering coefficient']:
        wavelength_var = True
    return wavelength_var

In [21]:

def get_files_info(files, var, path):
    
    files_info = {}
    files_info[var] = {}
    tqdm_iter = tqdm(files,bar_format= '{l_bar}{bar}|{n_fmt}/{total_fmt}',desc=f"    Creating information file ({len(files)})")
    for file in tqdm_iter:
        # open file
        try:
            ds = xr.open_dataset(file)
        except:
            continue

        # get resolution
        coverage = ds.time_coverage_resolution
        try:             
            file_resolution = coverages_dict[coverage]
        except:
            file_resolution = f'Unrecognised ({coverage})'
            
        file_start_date = ds.time_coverage_start
        file_end_date = ds.time_coverage_end
        file_variables = list(ds.data_vars.keys())
        files_info[var][file] = {}
        files_info[var][file]['resolution'] = file_resolution
        files_info[var][file]['start_date'] = file_start_date
        files_info[var][file]['end_date'] = file_end_date
        files_info[var][file]['variables'] = file_variables

    # create file
    datasets = {
        url: data
        for url, data in files_info[var].items()
    }
    if len(datasets) != 0:
        path_dir = os.path.dirname(path)
        if not os.path.exists(path_dir):
            os.makedirs(path_dir)
        with open(path, 'w') as file:
            yaml.dump(datasets, file, default_flow_style=False)
    
    return files_info

In [22]:

def get_data(files, var, actris_parameter, resolution):
    
    # combine datasets that have the same variable and resolution
    combined_ds_list = []
    metadata = {}
    metadata[resolution] = {}
    
    # get EBAS component
    ebas_component = variable_mapping[actris_parameter]['var']

    tqdm_iter = tqdm(files,bar_format= '{l_bar}{bar}|{n_fmt}/{total_fmt}',desc=f"    Reading data ({len(files)})")
    errors = {}
    for i, file in enumerate(tqdm_iter):
        # open file
        try:
            ds = xr.open_dataset(file)
        except:
            errors[file] = 'Error opening file'
            continue

        # get lowest level if tower height is in coordinates
        if 'Tower_inlet_height' in list(ds.coords):
            ds = ds.sel(Tower_inlet_height=min(ds.Tower_inlet_height.values), drop=True)

        # get data at desired wavelength if wavelength is in coordinates
        if 'Wavelength' in list(ds.coords):
            wavelength = int(re.findall(r'\d+', var)[0])
            if wavelength in ds.Wavelength.values:
                ds = ds.sel(Wavelength=wavelength, drop=True)
            else:
                errors[file] = f'- Data at {wavelength}nm could not be found'
                continue
        
        # assign station code as dimension
        ds = ds.expand_dims(dim={'station': [i]})

        # select data for that variable only
        unformatted_units = variable_mapping[actris_parameter]['units']
        if unformatted_units in units_dict.keys():
            units = units_dict[unformatted_units]
        else:
            errors[file] = f'- Units {unformatted_units} were not found in dictionary'
            continue
        units_var = f'{ebas_component}_{units}'
        possible_vars = [ebas_component, 
                         f'{ebas_component}_amean', 
                         units_var, 
                         f'{units_var}_amean']
        ds_var_exists = False
        for possible_var in possible_vars:
            if possible_var in ds:
                ds_var = ds[possible_var]
                ds_var_exists = True
                break

        # continue to next file if variable cannot be read
        if not ds_var_exists:
            errors[file] = f'- No variable name matches for {possible_vars}. Existing keys: {list(ds.data_vars)}'
            continue
            
        # save metadata
        for ghost_key, ebas_key in metadata_dict.items():
            # create key if it does not exist
            if ghost_key not in metadata[resolution].keys():
                metadata[resolution][ghost_key] = []

            # search value in var attrs
            if ebas_key in ds_var.attrs.keys():
                metadata[resolution][ghost_key].append(ds_var.attrs[ebas_key])
            # search value in ds attrs
            elif ebas_key in ds.attrs.keys():
                metadata[resolution][ghost_key].append(ds.attrs[ebas_key])
            # not found -> nan
            else:
                metadata[resolution][ghost_key].append(np.nan)

        # remove all attributes except units
        ds_var.attrs = {key: value for key, value in ds_var.attrs.items() if key == 'units'}

        # rename variable to BSC standards
        ds_var = ds_var.to_dataset(name=var)

        # append modified dataset to list
        combined_ds_list.append(ds_var)
    
    # show errors
    if len(errors) > 0:
        print('\nCollected errors:')
        for file, error in errors.items():
            print(f'{file} - Error: {error}')

    return combined_ds_list, metadata

In [23]:
def get_files_to_download(nonghost_root, target_start_date, target_end_date, resolution, var):

    base_dir = join(nonghost_root, 'actris/actris', resolution, var)
    paths = []
    current_date = copy.deepcopy(target_start_date)
    while current_date <= target_end_date:
        
        # save path
        path = f"{base_dir}/{var}_{current_date.strftime('%Y%m')}.nc"
        paths.append(path)

        # get following month
        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (current_date.month // 12)
        current_date = current_date.replace(year=next_year, month=next_month)

    return paths

In [24]:
def select_files_to_download(prov_start_time, nc_files_to_download):
    """ Returns the files that are not already downloaded. """
    # initialise list of non-downloaded files
    not_downloaded_files = []
    
    # get ssh user and password 
    env = dotenv_values(join(PROVIDENTIA_ROOT, ".env"))
    overwrite_choice = env.get("OVERWRITE")

    if nc_files_to_download:
        # get the downloaded and not downloaded files
        not_downloaded_files = list(filter(lambda x:not os.path.exists(x), nc_files_to_download))
        downloaded_files = list(filter(lambda x:os.path.exists(x), nc_files_to_download))
        
        # get the files that were downloaded before the execution
        downloaded_before_execution_files = list(filter(lambda x:prov_start_time > os.path.getctime(x), downloaded_files))

        # if there was any file downloaded before the execution    
        if downloaded_before_execution_files:
            # make the user choose between overwriting or not overwriting
            if overwrite_choice not in ['y','n']:
                # ask if user wants to overwrite
                while overwrite_choice not in ['y','n']:
                    overwrite_choice = input("\nThere are some files that were already downloaded in a previous download, do you want to overwrite them (y/n)? ").lower() 
                # ask if user wants to remember the decision
                remind_txt = None
                while remind_txt not in ['y','n']:
                    remind_txt = input("\nDo you want to remember your decision for future downloads (y/n)? ").lower() 
                # save the decision
                if remind_txt == 'y':
                    with open(join(PROVIDENTIA_ROOT, ".env"),"a") as f:
                        f.write(f"OVERWRITE={overwrite_choice}\n")
            # if user wants to overwrite then add the the files that were 
            # downloaded before the execution as if they were never download
            if overwrite_choice == 'y':
                not_downloaded_files += downloaded_before_execution_files
            # change overwritten files boolean to True to indicate that some files were ignored
            else:
                overwritten_files_flag = True

    return not_downloaded_files

In [25]:
# get providentia start time
prov_start_time = time.time()

for var in variables:

    nonghost_root = '/home/avilanov/data/providentia/obs/nonghost/'
    
    # get files that were already downloaded
    initial_check_nc_files = get_files_to_download(nonghost_root, target_start_date, target_end_date, resolution, var)
    files_to_download = select_files_to_download(prov_start_time, initial_check_nc_files)
    if not files_to_download:
        msg = f"\nFiles were already downloaded for {var} at {resolution} "
        msg += f"resolution between {target_start_date} and {target_end_date}."
        print(msg)  
        continue 
                
    actris_parameter = parameters_dict[var]
    path = get_files_path(var)
    
    # if file does not exist
    if not os.path.isfile(path):
        # get files information
        print(f'\nFile containing information of the files available in Thredds for {var} ({path}) does not exist, creating.')
        combined_data = get_files_per_var(var)
        all_files = combined_data[var]['files']
        files_info = get_files_info(all_files, var, path)
            
    # if file exists
    else:
        # ask if user wants to overwrite file
        origin_update_choice = None
        while origin_update_choice not in ['y','n']:
            origin_update_choice = input(f"\nFile containing information of the files available in Thredds for {var} ({path}) already exists. Do you want to update it (y/n)? ").lower() 
        if origin_update_choice == 'n':
            # get files information
            files_info = yaml.safe_load(open(os.path.join(CURRENT_PATH, path)))
            files_info = {k: v for k, v in files_info.items() if k.strip() and v}
        else:
            # get files information
            combined_data = get_files_per_var(var)
            all_files = combined_data[var]['files']
            files_info = get_files_info(all_files, var, path)
    
    # filter files by resolution and dates
    print('    Filtering files by resolution and dates...')
    files = []
    files_info = yaml.safe_load(open(os.path.join(CURRENT_PATH, path)))
    files_info = {k: v for k, v in files_info.items() if k.strip() and v}
    for file, attributes in files_info.items():
        if attributes["resolution"] == resolution:
            start_date = datetime.datetime.strptime(attributes["start_date"], "%Y-%m-%dT%H:%M:%S UTC")
            end_date = datetime.datetime.strptime(attributes["end_date"], "%Y-%m-%dT%H:%M:%S UTC")
            for file_to_download in files_to_download:
                file_to_download_yearmonth = file_to_download.split(f'{var}_')[1].split('.nc')[0]
                file_to_download_start_date = datetime.datetime.strptime(file_to_download_yearmonth, "%Y%m")
                file_to_download_end_date = datetime.datetime(file_to_download_start_date.year, file_to_download_start_date.month, 1) + relativedelta(months=1, seconds=-1)
                if file_to_download_start_date <= end_date and file_to_download_end_date >= start_date:
                    if file not in files:
                        files.append(file)
    
    if len(files) != 0:
            
        # get data and metadata for each file within period
        combined_ds_list, metadata = get_data(files, var, actris_parameter, resolution)
    
        # combine and create new dataset
        print('    Combining files...')
        try:
            combined_ds = xr.concat(combined_ds_list, 
                                    dim='station', 
                                    combine_attrs='drop_conflicts')
        except Exception as error:
            print(f'Error: Datasets could not be combined - {error}')
            if 'time' in str(error):
                for item in combined_ds_list:
                    print(item.time.values[0], item.time.values[1])
            continue
        
        # add metadata
        for key, value in metadata[resolution].items():
            if key in ['latitude', 'longitude']:
                value = [float(val) for val in value]
            elif key in ['altitude', 'measurement_altitude', 'sampling_height']:
                value = [float(val.replace('m', '').strip()) if isinstance(val, str) else val for val in value]
            combined_ds[key] = xr.Variable(data=value, dims=('station'))
    
        # add units for lat and lon
        # TODO: Check attrs geospatial_lat_units and geospatial_lon_units
        combined_ds.latitude.attrs['units'] = 'degrees_north'
        combined_ds.longitude.attrs['units'] = 'degrees_east'
    
        # add general attrs
        combined_ds.attrs['data_license'] = 'BSD-3-Clause. Copyright 2025 Alba Vilanova Cortezón'
        combined_ds.attrs['source'] = 'Observations'
        combined_ds.attrs['institution'] = 'Barcelona Supercomputing Center'
        combined_ds.attrs['creator_name'] = 'Alba Vilanova Cortezón'
        combined_ds.attrs['creator_email'] = 'alba.vilanova@bsc.es'
        combined_ds.attrs['application_area'] = 'Monitoring atmospheric composition'
        combined_ds.attrs['domain'] = 'Atmosphere'
        combined_ds.attrs['observed_layer'] = 'Land surface'
                
        # save data per year and month
        path = join(nonghost_root, f'actris/actris/{resolution}/{var}')
        if not os.path.isdir(path):
            os.makedirs(path, exist_ok=True)
        saved_files = 0
        for year, ds_year in combined_ds.groupby('time.year'):
            for month, ds_month in ds_year.groupby('time.month'):
                filename = f"{path}/{var}_{year}{month:02d}.nc"
                if filename in files_to_download:
                    combined_ds_yearmonth = combined_ds.sel(time=f"{year}-{month:02d}")
                    combined_ds_yearmonth = temporally_average_data(combined_ds_yearmonth, resolution, year, month, var)
    
                    # add title to attrs
                    extra_info = ''
                    wavelength_var = is_wavelength_var(actris_parameter)
                    if wavelength_var:
                        wavelength = int(re.findall(r'\d+', var)[0])
                        extra_info = f' at {wavelength}nm'
                    combined_ds_yearmonth.attrs['title'] = f'Surface {parameters_dict[var]}{extra_info} in the ACTRIS network in {year}-{month:02d}.'
    
                    # order attrs
                    custom_order = ['title', 'institution', 'creator_name', 'creator_email',
                                    'source', 'application_area', 'domain', 'observed_layer',
                                    'data_license']
                    ordered_attrs = {key: combined_ds_yearmonth.attrs[key] 
                                    for key in custom_order 
                                    if key in combined_ds_yearmonth.attrs}
                    combined_ds_yearmonth.attrs = ordered_attrs
    
                    # remove stations if all variable data is nan
                    previous_n_stations = len(combined_ds_yearmonth.station)
                    combined_ds_yearmonth = combined_ds_yearmonth.dropna(dim="station", subset=[var], how="all")
                    combined_ds_yearmonth = combined_ds_yearmonth.assign_coords(station=range(len(combined_ds_yearmonth.station)))
                    current_n_stations = len(combined_ds_yearmonth.station)
                    n_stations_diff = previous_n_stations - current_n_stations
                    if n_stations_diff > 0:
                        print(f'Data for {n_stations_diff} stations was removed because all data was NaN during {month}-{year}.')
                    
                    # save file
                    combined_ds_yearmonth.to_netcdf(filename)
    
                    # change permissions
                    os.system("chmod 777 {}".format(filename))
                    print(f"Saved: {filename}")
                    saved_files += 1
                    
        print(f'Total number of saved files: {saved_files}')


File containing information of the files available in Thredds for sconcno2 (/home/avilanov/software/Providentia/settings/internal/actris/files/sconcno2/files.yaml) already exists. Do you want to update it (y/n)?  n


    Filtering files by resolution and dates...


    Reading data (37): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|37/37


    Combining files...
Data for 7 stations was removed because all data was NaN during 1-2018.
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/actris/daily/sconcno2/sconcno2_201801.nc
Data for 7 stations was removed because all data was NaN during 2-2018.
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/actris/daily/sconcno2/sconcno2_201802.nc
Data for 7 stations was removed because all data was NaN during 3-2018.
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/actris/daily/sconcno2/sconcno2_201803.nc
Data for 7 stations was removed because all data was NaN during 4-2018.
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/actris/daily/sconcno2/sconcno2_201804.nc
Data for 7 stations was removed because all data was NaN during 5-2018.
Saved: /home/avilanov/data/providentia/obs/nonghost/actris/actris/daily/sconcno2/sconcno2_201805.nc
Data for 7 stations was removed because all data was NaN during 6-2018.
Saved: /home/avilanov/data/providentia/obs/no

In [26]:
end_time = time.time()

In [27]:
total_time = end_time - start_time
total_time

57.573200702667236

In [28]:
time.strftime("%H:%M:%S.{}".format(str(total_time % 1)[2:])[:15], time.gmtime(total_time))

'00:00:57.573200'

In [29]:
test_data = xr.open_dataset(f'/home/avilanov/data/providentia/obs/nonghost/actris/actris/{resolution}/{var}/{var}_201801.nc')
test_data

<xarray.Dataset> Size: 303kB
Dimensions:                                 (station: 30, time: 31)
Coordinates:
  * time                                    (time) datetime64[ns] 248B 2018-0...
  * station                                 (station) int64 240B 0 1 2 ... 28 29
Data variables: (12/26)
    station_reference                       (station) <U7 840B ...
    station_timezone                        (station) <U3 360B ...
    latitude                                (station) float64 240B ...
    longitude                               (station) float64 240B ...
    altitude                                (station) float64 240B ...
    sampling_height                         (station) float64 240B ...
    ...                                      ...
    contact_name                            (station) <U56 7kB ...
    contact_institution                     (station) <U56 7kB ...
    contact_email_address                   (station) <U12 1kB ...
    meta_update_stamp                       (station) <U23 3kB ...
    data_licence                            (station) <U44 5kB ...
    sconcno2                                (station, time) float64 7kB ...
Attributes:
    title:             Surface nitrogen dioxide mass concentration in the ACT...
    institution:       Barcelona Supercomputing Center
    creator_name:      Alba Vilanova Cortezón
    creator_email:     alba.vilanova@bsc.es
    source:            Observations
    application_area:  Monitoring atmospheric composition
    domain:            Atmosphere
    observed_layer:    Land surface
    data_license:      BSD-3-Clause. Copyright 2025 Alba Vilanova Cortezón